In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [16]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("h1", attrs={'class':'pdp_variation-name pl_headline100 pl_headline200--lg'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={"class": "js_pdp_price__retail-price__value pl_headline300"}).text[0:6]

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price


# Function to extract instalment payment
def instalment_price(soup):
    try:
        instalment_price = soup.find("strong", attrs={"class": "js_pdp_installment-plan-example__example-price"}).text + ' ' + new_soup.find("span", attrs={"class": "js_pdp_installment-plan-example__example-post-price"}).text

    except AttributeError:
        instalment_price = ""	

    return instalment_price

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("span", attrs={"class": "pl_headline50"}).text.strip()

    except AttributeError:
        available = "Not Available"	

    return available




In [17]:

if __name__ == '__main__':

    # add user agent 
    HEADERS = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36', 'Accept-Language': 'de-DE,de;q=0.9,en-US;q=0.8,en;q=0.7'})

    # The webpage URL
    URL = 'https://www.otto.de/suche/smartphone'

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'find_tile__productLink'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "instalment_price":[], "availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.otto.de" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['instalment_price'].append(instalment_price(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    otto_df = pd.DataFrame.from_dict(d)
    otto_df['title'].replace('', np.nan, inplace=True)
    otto_df = otto_df.dropna(subset=['title'])
    otto_df.to_csv("otto_data.csv", header=True, index=False)


In [18]:
otto_df

,title,price,instalment_price,availability
0,"Samsung Galaxy-Xcover-Pro Smartphone (16 cm/6,...","485,62","17,42 € mtl. in 36 Raten",lieferbar - in 1-2 Werktagen bei dir
1,"Samsung Galaxy S22 128 GB Smartphone (15,39 cm...","649,21","18,85 € mtl. in 48 Raten",lieferbar - in 1-2 Werktagen bei dir
2,"Motorola g13 Smartphone (16,56 cm/6,52 Zoll, 1...","133,00","12,15 € mtl. in 12 Raten",lieferbar - in 2-3 Werktagen bei dir
3,"Xiaomi Redmi A1 2+32 Smartphone (16,58 cm/6,52...","73,00","6,67 € mtl. in 12 Raten",lieferbar - in 4-5 Werktagen bei dir
4,"Samsung Galaxy A34 5G 128GB Smartphone (16,65 ...","331,00","16,44 € mtl. in 24 Raten",lieferbar - in 1-2 Werktagen bei dir
5,Xiaomi Redmi 12C 4GB+128GB Smartphone Handy (6...,"117,00","10,69 € mtl. in 12 Raten",lieferbar - in 4-5 Werktagen bei dir
6,"Xiaomi Redmi A2 2GB+32GB Smartphone (16,6 cm/6...","71,00","6,48 € mtl. in 12 Raten",lieferbar - in 5-6 Werktagen bei dir
7,"Xiaomi Redmi 12C 3GB+64GB Smartphone (17 cm/6,...","100,90","9,22 € mtl. in 12 Raten",lieferbar - in 2-3 Werktagen bei dir
8,Xiaomi Redmi Note 12 Pro 6+128GB Smartphone Ha...,"218,00","19,91 € mtl. in 12 Raten",lieferbar - in 4-5 Werktagen bei dir
